In [1]:
import gym
from gym import spaces
import numpy as np
import heuristicBot
import os
from stable_baselines.bench import Monitor
from stable_baselines.vec_env import DummyVecEnv

ModuleNotFoundError: No module named 'tensorflow.contrib'

Player representation : 1(X), 2(O)
Game grid : a 3x3 numpy array

In [6]:
def grid_score(grid):
    """
    Checks if the grid is completed.
    If yes and it is a draw(returns 0) or win(returns 1)
    else the game should continue(returns -1)
    """
    if 0 not in grid:
        return 0
    rcno = [0, 1, 2]
    # checking for rows and column condition
    for i in rcno:
        if grid[i, 0] == grid[i, 1] == grid[i, 2] != 0:
            return 1
        elif grid[0, i] == grid[1, i] == grid[2, i] != 0:
            return 1
    # checking for diagonals
    if grid[0, 0] == grid[1, 1] == grid[2, 2] != 0:
        return 1
    elif grid[0, 2] == grid[1, 1] == grid[2, 0] != 0:
        return 1
    return -1

In [9]:
def mark_piece(grid, index, player):
    """
    1. places a move by the RL agent
    2. check if invalid
    3. if not check if agent won or draw
    4. if not place move by opponent
    5. check if opponent won or draw
    6. return respective code at appropiate step
    returns -2:continue, -1:invalid move, 
            1:1 wins, 2:2 wins, 0:draw
    """
    if grid[index] != 0:
        return -1, grid
    else:
        grid[index] = player
        score = ttt.gridScore(grid)
        if score != -1 and score == 1:
            return player, grid
        elif score != -1:
            return 0, grid
        # opponent agent : a n-step-lookahead bot
        available_moves = [(i, j) for i in range(3)
                           for j in range(3) if grid[i, j] == 0]
        opp_player = (player % 2) + 1
        opp_move_idx = heuristicBot.nslAgent(
            2, grid, available_moves, opp_player)
        grid[opp_move_idx] = opp_player
        score = ttt.gridScore(grid)
        if score != -1 and score == 1:
            return opp_player, grid
        elif score != -1:
            return 0, grid
        return -2, grid

## Defining the Custom gym environment
**Rewards**
- 1 for winning
- -1 for loosing
- -10 for invalid move
- 1/9 for everything else

In [ ]:
class TicTacToeEnv(gym.Env):
    def __init__(self):
        """
        Define action and observation space
        They must be gym.spaces objects
        """
        super(TicTacToeEnv, self).__init__()
        self.action_space = spaces.Discrete(9)
        self.observation_space = spaces.Box(low=0, high=2, shape=(3,3), dtype=np.int)
        self.reward_range = (-10, 1)
        self.action_moves = [(i,j) for i in range(3) for j in range(3)]
        self.player = 1
        
        # defined else stable baselines throws error
        self.spec = None
        self.metadata = None


    def reset(self):
        """
        Reset the state of the environment to an initial state
        returns 
        eg.
            return self.state
        """
        self.grid = np.zeros((3, 3))

        return self.grid


    def step(self, action):
        """
        Execute one time step within the environment
        returns next state
        eg. 
            modify state
            return self.state, reward, done, info
        """
        score, self.grid = mark_piece(self.grid, self.action_moves[action], self.player)

        # reward
        done = True
        if score == -1:
            reward = -10
        elif score == 1:
            reward = 1
        elif score == 2:
            reward = -1
        elif score == -2:
            done = False
            reward = 1/9

        # check if grid ended

        # info for debugginh
        info = {}
        
        return self.grid, reward, done, info


    def render(self, mode='human', close=False):
        # Render the environment to the screen
        pass
